# import libraries 

In [14]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import math 

# import data 

In [15]:
!wget ?-O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
!unzip -o -j moviedataset.zip 

http://?-O: Invalid host name.
--2023-02-21 15:17:29--  http://moviedataset.zip/
Resolving moviedataset.zip (moviedataset.zip)... 78.47.226.171
Connecting to moviedataset.zip (moviedataset.zip)|78.47.226.171|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4855 (4.7K) [text/html]
Saving to: ‘index.html.1’

index.html.1        100%[===================>]   4.74K  --.-KB/s    in 0.008s  

2023-02-21 15:17:46 (608 KB/s) - ‘index.html.1’ saved [4855/4855]

--2023-02-21 15:17:46--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... failed: Connection refused.
FINISHED

In [37]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

# data cleaning 

In [38]:
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')
movies['title'] = movies['title'].apply(lambda x: x.strip())
movies.head()

/tmp/ipykernel_79622/1226832952.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


In [39]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [40]:
ratings = ratings.drop('timestamp', 1)
ratings.head()

/tmp/ipykernel_79622/1184197411.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings = ratings.drop('timestamp', 1)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


# Collaborative Filtering

In [45]:
def find_input_id (movies,rating_movies):
    return  movies[movies['title'].isin(rating_movies['title'].tolist())]

In [47]:
def find_users_ratings (ratings , user_movies):
    return ratings[ratings['movieId'].isin(user_movies['movieId'].tolist())]

In [33]:
def collaborative_filtering (user_rating , movies, ratings):
    
    inputId = find_input_id(movies,user_rating)
    inputMovies = pd.DataFrame(userInput)
    inputMovies = pd.merge(inputId, inputMovies)
    
    users_rating = find_users_ratings(ratings,inputMovies)
    
    

IndentationError: expected an indented block (2706010398.py, line 3)